# On-Target Threshold
Determine the value of hyperparameter `cnfg.ON_TARGET_THRESHOLD`, which is used to decide if a gaze sample / fixation / visit is "on-target" or not.

In [12]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import config as cnfg

# pio.renderers.default = "notebook"
pio.renderers.default = "browser"

### Read data

In [13]:
from analysis.pipeline.full_pipeline import read_saved_data

_targets, _actions, _metadata, idents, fixations, _visits = read_saved_data()

### Show Data:
#### (1) Distances-from-Target when subject performed the identification action

In [14]:
percentiles = [0.5, 0.75, 0.85, 0.9, 0.95, 0.99,]

not_misses = idents.loc[idents["identification_category"] != "miss"]
dist_summary = (
    pd.concat([
        not_misses["distance_dva"].describe(percentiles).rename("all"),
        not_misses.groupby("subject")["distance_dva"].describe(percentiles).T,
    ], axis=1)
).T

dist_summary

count      mean       std       min       50%       75%       85%  \
all  1358.0  0.802777  1.579489  0.005250  0.531129  0.831108  1.016362   
2     112.0  0.887389  1.378940  0.005250  0.638451  0.945330  1.132480   
3     110.0  0.522337  0.318907  0.016496  0.446273  0.649872  0.818994   
12    111.0  1.377363  2.263406  0.160329  0.863651  1.245024  1.640418   
13     99.0  0.782881  1.451985  0.073701  0.538058  0.731862  0.912429   
14    104.0  1.307629  3.840592  0.032188  0.370250  0.633209  0.733201   
15     98.0  0.809266  1.268136  0.085205  0.675779  0.865602  1.059637   
16     90.0  0.423305  0.247078  0.043483  0.350909  0.640706  0.747622   
17    101.0  0.764956  1.272487  0.065453  0.567267  0.825397  1.034370   
18    123.0  0.442788  0.809888  0.057758  0.359878  0.493016  0.569625   
19    116.0  0.792266  0.767282  0.108211  0.701126  1.014256  1.145919   
20     91.0  0.741688  0.868017  0.085384  0.591672  0.899172  1.008872   
21    105.0  0.588888  0.932753  0.082529  0.434717  0.559546  0.726120   
22     98.0  0.985527  1.177005  0.078698  0.787950  1.084606  1.359647   

          90%       95%        99%        max  
all  1.173581  1.440956   8.195754  23.297429  
2    1.240759  1.367437   8.195211   9.435844  
3    0.953997  1.151773   1.507079   1.723973  
12   1.947023  4.304638   9.813110  19.109826  
13   1.155491  1.408013   6.124180  13.597367  
14   0.991192  5.101486  21.035096  23.297429  
15   1.202942  1.377850   2.032327  12.765376  
16   0.778273  0.858737   0.915361   0.967981  
17   1.153490  1.318819   3.755980  12.600789  
18   0.628443  0.778915   0.995772   9.073654  
19   1.215355  1.350801   1.971149   7.938850  
20   1.099905  1.212935   6.045439   6.230239  
21   0.804119  0.929096   5.190986   7.661779  
22   1.483505  1.680721   7.939592   8.176036

In [15]:
not_misses

subject  trial   target identification_category     time  to_trial_end  \
0           2      1  target0                     hit  10280.0       10441.0   
1           2      1  target1                     hit   6976.0       13745.0   
2           2      2  target0                     hit  16368.0        3929.0   
4           2      3  target0                     hit   3614.0       16859.0   
5           2      3  target1                     hit   7654.0       12819.0   
...       ...    ...      ...                     ...      ...           ...   
1925       22     58  target0                     hit    637.0       11828.0   
1926       22     58  target1             false_alarm   4253.0        8212.0   
1928       22     59  target0             false_alarm   2515.0        6876.0   
1930       22     59  target1             false_alarm    747.0        8644.0   
1933       22     60  target1                     hit   5099.0        7220.0   

      distance_px  distance_dva     left_x    left_y  left_pupil    right_x  \
0        7.911587      0.202457        NaN       NaN         NaN   530.2656   
1       32.018560      0.819353   887.8080  631.9944     3.63455   847.9680   
2       20.564596      0.526247   500.0448  527.0724     3.33818   492.2688   
4        8.747013      0.223835   704.2368  285.0120     3.52066   678.4128   
5       31.524358      0.806706  1518.3552  425.3688     3.61078  1529.0304   
...           ...           ...        ...       ...         ...        ...   
1925    35.367994      0.935233   920.0256  478.3428     3.49547   865.8816   
1926    38.883752      1.028200  1391.4432  707.0760     3.74825  1366.4832   
1928    38.162648      1.009132   463.6224  644.8032     3.93939   428.2560   
1930    61.099378      1.615647  1018.0992  657.9900     3.45514   954.6624   
1933    28.702213      0.758971  1474.2336  373.8528     3.96553  1453.0944   

       right_y  right_pupil  
0     218.0952      3.66040  
1     628.5276      3.66786  
2     541.9440      3.36696  
4     280.3356      3.60327  
5     435.5316      3.63414  
...        ...          ...  
1925  459.0000      3.67410  
1926  687.7440      3.87274  
1928  616.1616      4.05275  
1930  636.3252      3.56114  
1933  340.4376      4.09567  

[1358 rows x 14 columns]

In [16]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, shared_yaxes=False,
)

# top: distribution across all subjects
fig.add_trace(
    row=1, col=1, trace=go.Violin(
        y0="distance", x=not_misses["distance_dva"],
        name="All Subjects", legendgroup="All Subjects",
        text=not_misses.apply(
            lambda row: f"Subject: {row['subject']}<br>"
                        f"Trial: {row['trial']}<br>"
                        # f"Target: {row['target']}<br>"
                        f"Distance: {row['distance_dva']:.2f} DVA",
            axis=1
        ),
        marker=dict(color=cnfg.get_discrete_color("all")),
        width=1.75, orientation="h", side="positive", spanmode='hard',
        box=dict(visible=False),
        meanline=dict(visible=True),
        points="all", pointpos=-0.5,
        showlegend=True, hoverinfo="x+y+text",

    )
)

# bottom: distribution per subject
for subj_id in not_misses[cnfg.SUBJECT_STR].unique():
    subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
    subj_data = not_misses[not_misses[cnfg.SUBJECT_STR] == subj_id]
    texts = subj_data.apply(
        lambda row: f"{subj_string}<br>"
                    f"Trial: {row['trial']}<br>"
                    # f"Target: {row['target']}<br>"
                    f"Distance: {row['distance_dva']:.2f} DVA",
        axis=1
    )
    fig.add_trace(
        row=2, col=1, trace=go.Violin(
            y0="distance", x=subj_data["distance_dva"],
            text=texts,
            name=subj_string, legendgroup=subj_string,
            marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=True, hoverinfo="x+y+text"
        )
    )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1200, height=675,
    title=dict(text="Distance on Identification-Action", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()

#### (2) Distances-from-Target across all fixations
#### (2) Distances-from-Target during identification-fixations

In [17]:
percentiles = [0.05, 0.25, 0.5, 0.75, 0.9, 0.95]

dva_cols = [col for col in fixations.columns if col.endswith("distance_dva")]
min_dists = pd.concat([fixations[["subject", "trial", "eye", "event"]], fixations[dva_cols].min(axis=1).rename("distance")], axis=1)
fixation_dist_summary = (
    pd.concat([
        min_dists["distance"].describe(percentiles).rename("all"),
        min_dists.groupby("subject")["distance"].describe(percentiles).T,
    ], axis=1)
).T

print("All Fixations:")
fixation_dist_summary

All Fixations:


count      mean       std       min        5%       25%       50%  \
all  127122.0  8.476323  5.224281  0.005305  0.815494  4.543053  8.023953   
2     12482.0  7.863880  4.989668  0.014836  0.670461  3.992184  7.518766   
3     10175.0  8.522620  5.305019  0.005305  0.790146  4.685162  8.029637   
12    11733.0  8.639695  5.282517  0.021815  0.984320  4.770353  8.085927   
13     8135.0  8.547193  5.522265  0.041144  0.728294  4.449000  8.025514   
14     9513.0  9.016266  5.347366  0.039167  0.842865  4.987896  8.498420   
15     7616.0  8.245533  5.062607  0.049419  0.783031  4.280422  7.964702   
16     9973.0  8.743079  4.964180  0.007104  0.937009  4.985892  8.433800   
17     7815.0  8.095184  5.138432  0.015262  0.880142  4.184559  7.579997   
18    11202.0  8.231489  5.129308  0.014692  0.722300  4.453289  7.606037   
19     7266.0  8.019145  5.344337  0.022844  0.757255  3.716248  7.650134   
20    10349.0  8.968774  5.251652  0.006736  0.932083  5.034295  8.673973   
21    11011.0  8.335090  5.029629  0.007058  0.668841  4.535895  7.952605   
22     9852.0  8.896819  5.471864  0.034739  1.031145  4.776186  8.382631   

           75%        90%        95%        max  
all  11.746939  15.304024  17.842813  43.422000  
2    10.957089  14.360563  16.910656  28.950040  
3    11.588239  15.361410  18.202060  31.530082  
12   11.792952  15.344020  18.479218  31.039929  
13   11.798203  15.770773  18.720618  32.371653  
14   12.553002  16.191683  18.383610  43.422000  
15   11.599680  15.154948  17.057289  39.595361  
16   12.005936  15.403083  17.687778  27.668284  
17   11.337777  14.953332  17.188340  29.743946  
18   11.328712  15.024183  17.708345  30.078966  
19   11.408590  14.759613  17.316979  31.603614  
20   12.255925  15.813083  18.041435  31.435273  
21   11.610980  14.730150  17.051092  27.860347  
22   12.230619  15.962495  19.012058  32.892468

##### find identification fixations
fixations where either:
- the subject performed an identification action during the fixation
- the subject performed an identification action immediately after the fixation

In [18]:
fixs_with_ident_time = fixations.copy()
fixs_with_ident_time["target"] = fixs_with_ident_time[dva_cols].idxmin(axis=1).str.replace("_distance_dva", "")
fixs_with_ident_time["distance_dva"] = fixs_with_ident_time[dva_cols].min(axis=1)
fixs_with_ident_time = (
    fixs_with_ident_time
    .drop(columns=[col for col in fixs_with_ident_time.columns if "_distance_" in col])
    .merge(
        idents.loc[
            idents["identification_category"] == "hit", ["subject", "trial", "target", "time"]
        ], on=["subject", "trial", "target"], how="left"
    )
)

fixs_with_ident_time.loc[:, "is_during"] = (fixs_with_ident_time["start_time"] <= fixs_with_ident_time["time"]) & (fixs_with_ident_time["time"] <= fixs_with_ident_time["end_time"])

fixs_with_ident_time.loc[:, "end_to_ident_diff"] = fixs_with_ident_time["time"] - fixs_with_ident_time["end_time"]
fixs_with_ident_time.loc[:, "is_immediately_preceding"] = False
immediately_preceding_idxs = (
    fixs_with_ident_time
    .loc[(0 <= fixs_with_ident_time["end_to_ident_diff"]) & (fixs_with_ident_time["end_to_ident_diff"] <= 1000)]    # max 1 sec
    .groupby(["subject", "trial", "eye", "target"])["end_to_ident_diff"]
    .idxmin()
    .values
)
fixs_with_ident_time.loc[immediately_preceding_idxs, "is_immediately_preceding"] = True
# fixs_with_ident_time.drop(columns=["end_to_ident_diff"], inplace=True)

In [19]:
ident_fixs = fixs_with_ident_time.loc[fixs_with_ident_time["is_during"]]
ident_fixs_dist_summary = (
    pd.concat([
        ident_fixs["distance_dva"].describe(percentiles).rename("all"),
        ident_fixs.groupby("subject")["distance_dva"].describe(percentiles).T,
    ], axis=1)
).T

print("Identification Fixations:")
ident_fixs_dist_summary

Identification Fixations:


count      mean       std       min        5%       25%       50%  \
all  2227.0  0.491641  0.273467  0.006736  0.112503  0.290022  0.449929   
2     160.0  0.530030  0.288054  0.038549  0.131345  0.288115  0.489816   
3     198.0  0.442627  0.227006  0.014953  0.107188  0.298885  0.402673   
12    136.0  0.527307  0.262566  0.021815  0.142911  0.335189  0.495572   
13    172.0  0.530976  0.271928  0.041144  0.150215  0.347978  0.477650   
14    183.0  0.392083  0.215216  0.042179  0.106613  0.222227  0.367571   
15    160.0  0.577590  0.254450  0.049419  0.207063  0.375086  0.567302   
16    180.0  0.392280  0.219487  0.039001  0.082794  0.215285  0.348203   
17    166.0  0.519017  0.237223  0.027277  0.180389  0.329269  0.510460   
18    233.0  0.445753  0.270403  0.014692  0.087837  0.264321  0.404707   
19    163.0  0.561783  0.290230  0.022844  0.178646  0.346355  0.498303   
20    148.0  0.496805  0.260989  0.006736  0.110077  0.315265  0.445913   
21    198.0  0.404452  0.213734  0.008924  0.100978  0.232080  0.390134   
22    130.0  0.687885  0.404076  0.044427  0.187189  0.408791  0.601051   

          75%       90%       95%       max  
all  0.656058  0.868502  0.969499  2.338724  
2    0.765306  0.941329  1.006351  1.297531  
3    0.557754  0.731696  0.870882  1.513686  
12   0.708934  0.906566  0.972123  1.217615  
13   0.680736  0.906162  1.056087  1.344511  
14   0.530472  0.699638  0.758516  1.366193  
15   0.782218  0.913637  0.969219  1.108418  
16   0.565713  0.710165  0.766271  0.974541  
17   0.669308  0.823450  0.926374  1.127256  
18   0.569300  0.849090  0.984403  1.543633  
19   0.760929  0.958697  1.070595  1.620089  
20   0.651578  0.871479  0.946170  1.246313  
21   0.548533  0.678617  0.788098  1.073168  
22   0.915530  1.116384  1.520467  2.338724

In [20]:
preceding_fixs = fixs_with_ident_time.loc[fixs_with_ident_time["is_immediately_preceding"]]
preceding_fixs_dist_summary = (
    pd.concat([
        preceding_fixs["distance_dva"].describe(percentiles).rename("all"),
        preceding_fixs.groupby("subject")["distance_dva"].describe(percentiles).T,
    ], axis=1)
).T

print("Preceding Identification Fixations:")
preceding_fixs_dist_summary

Preceding Identification Fixations:


count      mean       std       min        5%       25%       50%  \
all  2201.0  1.322059  1.789382  0.007757  0.205502  0.498251  0.867291   
2     165.0  1.522955  1.899072  0.038814  0.199154  0.497136  0.869727   
3     198.0  1.712710  2.222550  0.026879  0.191977  0.516824  0.940970   
12    133.0  1.534406  1.712427  0.028722  0.347126  0.596772  1.033246   
13    170.0  0.941220  0.639363  0.057448  0.247683  0.523218  0.813986   
14    170.0  1.601367  2.516978  0.039167  0.252464  0.496040  0.873392   
15    159.0  1.782876  2.845960  0.076795  0.222818  0.540849  0.883613   
16    172.0  1.495051  2.322052  0.092160  0.203420  0.547694  0.926373   
17    166.0  1.801119  1.980107  0.121700  0.319646  0.700139  1.219830   
18    231.0  0.883785  0.758658  0.036474  0.143061  0.398401  0.704171   
19    158.0  1.332309  1.718917  0.039494  0.241194  0.633475  1.027331   
20    147.0  0.935069  0.638826  0.111683  0.245557  0.498330  0.795514   
21    202.0  0.807764  0.733133  0.007757  0.139119  0.351902  0.599459   
22    130.0  0.986434  0.609526  0.048736  0.235299  0.542742  0.911576   

          75%       90%       95%        max  
all  1.433921  2.415173  4.014984  19.686305  
2    1.714575  4.111322  5.627576  10.295021  
3    1.818814  4.324278  5.232185  14.432919  
12   1.668116  2.593659  4.868113   9.493544  
13   1.232215  1.599860  1.973387   5.284142  
14   1.704897  3.367300  4.496134  16.502108  
15   1.885028  3.485988  5.057811  16.967226  
16   1.464096  2.931335  4.588137  19.686305  
17   1.950061  3.865790  7.422178  10.451534  
18   1.033111  1.735507  2.478636   4.267241  
19   1.509759  1.909214  2.760646  14.663008  
20   1.193821  1.767236  2.154732   3.477422  
21   1.007726  1.612743  2.023192   5.094546  
22   1.346215  1.669271  1.882668   4.070489

In [21]:
column_titles = ["All Fixations", "Co-Occurring with Identification", "Preceding Identification"]
fig = make_subplots(
    rows=2, cols=len(column_titles), column_titles=column_titles,
    shared_xaxes=True, shared_yaxes=False,
)

for c in range(len(column_titles)):
    if c == 0:
        data = fixs_with_ident_time
    elif c == 1:
        data = fixs_with_ident_time[fixs_with_ident_time["is_during"]]
    elif c == 2:
        data = fixs_with_ident_time[fixs_with_ident_time["is_immediately_preceding"]]
    else:
        raise ValueError(f"Unexpected column index {c}.")
    # top: distribution across all subjects
    fig.add_trace(
        row=1, col=c+1, trace=go.Violin(
            y0="distance", x=data["distance_dva"],
            name="All Subjects", legendgroup="All Subjects",
            text=data.apply(
                lambda row: f"Subject: {row['subject']}<br>"
                            f"Trial: {row['trial']}<br>"
                            f"Target: {row['target']}<br>"
                            f"Distance: {row['distance_dva']:.2f} DVA",
                axis=1
            ),
            marker=dict(color=cnfg.get_discrete_color("all")),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=c==0, hoverinfo="x+y+text",

        )
    )
    # bottom: distribution per subject
    for subj_id in data[cnfg.SUBJECT_STR].unique():
        subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
        subj_data = data[data[cnfg.SUBJECT_STR] == subj_id]
        texts = subj_data.apply(
            lambda row: f"{subj_string}<br>"
                        f"Trial: {row['trial']}<br>"
                        f"Target: {row['target']}<br>"
                        f"Distance: {row['distance_dva']:.2f} DVA",
            axis=1
        )
        fig.add_trace(
            row=2, col=c+1, trace=go.Violin(
                y0="distance", x=subj_data["distance_dva"],
                text=texts,
                name=subj_string, legendgroup=subj_string,
                marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
                width=1.75, orientation="h", side="positive", spanmode='hard',
                box=dict(visible=False),
                meanline=dict(visible=True),
                points="all", pointpos=-0.5,
                showlegend=c==0, hoverinfo="x+y+text"
            )
        )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1400, height=650,
    title=dict(text="Distance on Fixations", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()